# Explain diamonds model

## Load libraries and data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.ion()
import seaborn as sns
# import os

## Prepare data for descriptive analysis

In [ ]:
# Function that prepares a pandas df line by line
def preprocess(df):
    yes_no = {True: "ja", False: "nein"}
    return df.assign(xx = (bool).map(yes_no))

df = preprocess(df)

# Important variable groups
response = "…"
num_vars = ["some cov"]

ordinal_vars = ["more covs"]

features = num_vars + ordinal_vars

## Describe data

### Univariate categorical

In [ ]:
g = sns.FacetGrid(df[[response] + ordinal_vars].melt(), col="variable", 
                  sharex=False, sharey=False, col_wrap=3)
g = g.map(sns.countplot, "value")
g.set_xticklabels(rotation=30, ha="right")
g.set_titles(row_template = '{row_name}', col_template = '{col_name}')
g.fig.tight_layout()

### Univariate numeric

In [ ]:
g = sns.FacetGrid(df[num_vars].melt(), col="variable", 
                  sharex=False, sharey=False, col_wrap=3)
g = g.map(sns.distplot, "value", kde=False)
g.set_titles(row_template = '{row_name}', col_template = '{col_name}')
g.fig.tight_layout()

### Missing values

In [ ]:
df.isna().sum().sort_values(ascending=False).head(10)

### Associations with response

In [ ]:
# Grouped categoric
fig, axes = plt.subplots(3, 3, figsize=(12, 10))

for i, v in enumerate(ordinal_vars):
    sns.countplot(v, hue=response, data=df, ax=axes[i // 3][i % 3])
fig.tight_layout()

# Grouped numeric
dflong = df[num_vars + [response]].melt(id_vars=[response])
g = sns.catplot(response, "value", sharex=False, sharey=False, kind="box", 
                data=dflong, col="variable", col_wrap=2, showfliers=False)
g.set_titles(row_template = '{row_name}', col_template = '{col_name}');

## Prepare data for modeling

In [ ]:
# Safe preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

# Define transformer (note: OrdinalEncoder does not allow for missing values)
imputer = make_column_transformer((SimpleImputer(strategy="median"), num_vars),
                                  (SimpleImputer(strategy="constant", fill_value="most_frequent"), ordinal_vars))
ordinal_encoder = make_column_transformer((OrdinalEncoder(), ordinal_vars))

# Response & features
y = df[response]
X = df[features]

# Apply imputer
X[features] = imputer.fit_transform(X)

# Branch further preprocessing from interpretation
X_display = X.copy()

print("Data structure for interpretation:\n\n", X_display.head())

# Integer coding
X[ordinal_vars] = ordinal_encoder.fit_transform(X)

## Modeling

### Data split

In [ ]:
# Data split
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# create a train/test split
X_display_train, X_display_test, X_train, X_test, y_train, y_test = train_test_split(
    X_display, X, y, test_size=0.2, random_state=7, shuffle=True)

dtrain = lgb.Dataset(X_train, 
                     label=y_train, 
                     categorical_feature=["cat feature"], 
                     free_raw_data=False)

### Tune model

In [ ]:
# Cross-validation for iterative parameter selection
if False:
    params = {
        "learning_rate": 0.1,
        "num_leaves": 10,
        'min_child_samples': 20,
        'min_split_gain': 0.0001,
        'subsample': 0.9,
        'bagging_freq': 1,
        'colsample_bytree': 0.7,
        'reg_alpha': 2,
        'reg_lambda': 2,
        "verbose": 2,
        "objective": "binary",
        "metric": "binary_logloss",
        "nthread": 8
    }

    # Parameters where tuned in this order - should add randomized grid search after preselection
    key = "num_leaves"; varying = [7, 15, 31, 63]

    for i, v in enumerate(varying):
        params[key] = v

        cv_results = lgb.cv(
            params,
            dtrain,
            num_boost_round=1000,
            nfold=5,
            early_stopping_rounds=20)
        logloss = cv_results['binary_logloss-mean']
        print(v, f"({len(logloss)}):\t", logloss[-1])

### Fit model

In [ ]:
# Reasonable params
params = {
    "learning_rate": 0.1,
    "num_leaves": 10,
    'min_child_samples': 20,
    'min_split_gain': 0.0001,
    'subsample': 0.9,
    'bagging_freq': 1,
    'colsample_bytree': 0.7,
    'reg_alpha': 2,
    'reg_lambda': 2,
    "verbose": 2,
    "objective": "binary",
    "metric": "binary_logloss",
    "nthread": 8
}

# Fit the model
model = lgb.train(params, dtrain, 530)

### Performance

ROC (with AUC) on both training and test data.

In [ ]:
# Performance (ROC Plot)
from sklearn.metrics import roc_auc_score, roc_curve
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

# ROC 
fpr_train, tpr_train, _ = roc_curve(y_train, pred_train)
fpr_test, tpr_test, _ = roc_curve(y_test, pred_test)

fig, axis = plt.subplots(figsize=(6, 3))
axis.plot(fpr_train, tpr_train, label='Train AUC: {:.2f}'.format(roc_auc_score(y_train, pred_train)))
axis.plot(fpr_test, tpr_test, label='Test AUC: {:.2f}'.format(roc_auc_score(y_test, pred_test)))
axis.legend();

## Explain model

In [ ]:
import shap

# print the JS visualization code to the notebook
shap.initjs()

# Number of SHAP decompositions
N_SHAP = 2000 

# Initialize tree explainer
explainer = shap.TreeExplainer(model)

# Calculate N_SHAP decompositions
shap_values = explainer.shap_values(X_train[:N_SHAP])[1]

### Variable importance

In [ ]:
shap.summary_plot(shap_values, X_train[:N_SHAP], plot_type="bar")

### Effects plots

In [ ]:
for name in sorted_cols:
    shap.dependence_plot(name, shap_values, X_train[:N_SHAP], display_features=X_display_train[:N_SHAP], 
                         show=False, alpha=0.1, dot_size=4, x_jitter=0.1, 
                         xmin="percentile(1)", xmax="percentile(99)")
    plt.title(f"Dependence plot - {name}", fontdict={'fontsize': 15})
    plt.show()